In [5]:
import numpy as np
import torch, torch.nn as nn
import gymnasium as gym

In [2]:
gym.register(id='Dragonsweeper-v0', entry_point="Environment:DragonSweeperEnv")
device = torch.device("cpu")

In [3]:
env = gym.make("Dragonsweeper-v0", render_mode=None)

In [8]:
def flat_obs(obs, device=device):
    board = torch.tensor(obs['board'], dtype=torch.float32, device=device).flatten()
    player = torch.tensor(obs['player'], dtype=torch.float32, device=device).flatten()
    return torch.cat((board, player), dim=0)

In [9]:
obs_size = len(flat_obs(env.reset()[0])) # TODO: Remove this before end, yucky pointless reset
channel_size = 256
num_actions = env.action_space.n

In [10]:
num_actions

np.int64(131)

In [11]:
simp_net = torch.nn.Sequential(
    nn.Linear(obs_size, channel_size),
    nn.LayerNorm(channel_size, channel_size),
    nn.ReLU(),
    nn.Linear(channel_size, channel_size),
    nn.LayerNorm(channel_size, channel_size),
    nn.ReLU(),
    nn.Linear(channel_size, num_actions),
    nn.LayerNorm(num_actions, num_actions),
    nn.ReLU(),
)

In [39]:
optimizer = torch.optim.Adam(params=model.parameters())

In [40]:
obs, _ = env.reset()

In [41]:
losses = []
actions = []

In [42]:
eps = 1e-2

In [43]:
for episode in range(1000):
    logits = model(flat_obs(obs))
    probs = nn.functional.softmax(logits, dim=0)
    probs = (1-eps) * probs + eps/num_actions
    action = torch.multinomial(probs, num_samples=1).item()
    obs, reward, terminated, truncated, _ = env.step(action)
    loss = -(torch.log_softmax(logits, dim=0)[action]) * reward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if terminated or truncated:
        actions.append(action)
        obs, _ = env.reset()